In [67]:
from valurap2 import commands
from valurap2 import buf_commands
import time
import imp
imp.reload(buf_commands)
imp.reload(commands)

p = commands.S3GPort()
p.S3G_OUTPUT(63, 33)
v = p.S3G_INPUT(63)
display(v)

cb = buf_commands.CommandBuffer()
cb.hw_reset()
cb.BUF_DONE()

a = p.S3G_INPUT(cb.IN_MOTOR1_X)
print("X1:", a)
a = p.S3G_INPUT(cb.IN_MOTOR2_X)
print("X2:", a)

p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
while True:
    a = p.S3G_INPUT(cb.IN_BE_STATUS)
    print("BE_STATUS", a)
    if cb.extract_field(cb.IN_BE_STATUS_BUSY, a) == 0:
        break
    time.sleep(0.1)

a = p.S3G_INPUT(cb.IN_MOTOR1_X)
print("X1:", a)
a = p.S3G_INPUT(cb.IN_MOTOR2_X)
print("X2:", a)
a = p.S3G_INPUT(cb.IN_APG_STATUS)
print("APG_STATUS", a)


33

X1: 0
X2: 0
BE_STATUS 0
X1: 0
X2: 0
APG_STATUS 0


In [2]:

s_t = 0.001
f = 50000000
sb = 40

n = 400 * 32
t = 1

v = n * (2 ** sb) / f / t
v1 = v * 2
n_s = t / s_t
a = v1 / n_s
dt = f * s_t
v, a, dt, n_s

(281474976.710656, 562949.9534213119, 50000.0, 1000.0)

In [65]:
import time
from valurap2.buf_commands import CommandBuffer
cb = CommandBuffer(debug=True)

cb.BUF_OUTPUT(0, 50000)        # step time 0.001s
cb.BUF_OUTPUT(2, 40)           # step_bit = 40
cb.BUF_OUTPUT(3, 100)          # pre_n
cb.BUF_OUTPUT(4, 400)          # pulse_n
cb.BUF_OUTPUT(5, 500)          # post_n
cb.BUF_OUTPUT(7, 0x80818080)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(13, 500000)       # ES_TIMEOUT 10ms

cb.BUF_OUTPUT(1, 5000)              # steps_val = 2000 (2 seconds)
cb.write_param(0, cb.PARAM_STATUS, cb.PARAM_STATUS_ENABLE | cb.PARAM_STATUS_TARGET_V)           # CH0.STATUS=3 (enable + target_v)
cb.write_param(0, cb.PARAM_V_OUT, 0)
cb.write_param(0, cb.PARAM_A , 500000)
cb.write_param(0, cb.PARAM_J, 0)
cb.write_param(0, cb.PARAM_JJ, 0)
cb.write_param(0, cb.PARAM_TARGET_V, 50000000)
cb.write_param(0, cb.PARAM_ABORT_A, 500000)
for ch in range(1, 8):
    cb.write_param(ch, 0, 0)

cb.BUF_STB(cb.STB_ASG_START)   # Start ASG cycles
cb.BUF_WAIT_ALL(cb.INT_ASG_LOAD)  # Wait for APG ready for new segment
cb.BUF_CLEAR(cb.INT_ASG_LOAD)     # Clear pending int

cb.BUF_OUTPUT(1, 3000)           # steps_val = 3000 (3 seconds)
cb.write_param(0, cb.PARAM_STATUS, cb.PARAM_STATUS_ENABLE | cb.PARAM_STATUS_TARGET_V)           # CH0.STATUS=3 (enable + target_v)
cb.write_param(0, cb.PARAM_A , -500000)
cb.write_param(0, cb.PARAM_TARGET_V, 5000000)

cb.BUF_STB(cb.STB_ASG_LOAD_DONE)  # Inform ASG, next segment setup is done
cb.BUF_WAIT_ALL(cb.INT_ASG_LOAD)
cb.BUF_CLEAR(cb.INT_ASG_LOAD)

cb.BUF_OUTPUT(1, 1000)      # steps_val = 1000 (1 second)

cb.write_param(0, cb.PARAM_STATUS, cb.PARAM_STATUS_ENABLE | cb.PARAM_STATUS_TARGET_V)           # CH0.STATUS=3 (enable + target_v)
cb.write_param(0, cb.PARAM_A , -500000)
cb.write_param(0, cb.PARAM_TARGET_V, 0)

cb.BUF_STB(cb.STB_ASG_LOAD_DONE)
cb.BUF_WAIT_ALL(cb.INT_ASG_LOAD)
cb.BUF_CLEAR(cb.INT_ASG_LOAD)

cb.BUF_OUTPUT(1, 0)         # steps_val = 0 - End of path
cb.BUF_STB(cb.STB_ASG_LOAD_DONE)
cb.BUF_WAIT_ALL(cb.INT_ASG_DONE)  # Wait for ASG done
cb.BUF_CLEAR(cb.INT_ASG_DONE)

cb.BUF_OUTPUT(7, 0x00000000) # disable 1 and 2 motors
cb.BUF_DONE(0)                # End of program for BE

p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(cb.STB_ASG_ABORT)
p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
p.S3G_STB(cb.STB_ES_UNLOCK)

oa, ob, oc, od1, od2 = None, None, None, -1, -1
pd1 = -1
pd2 = -1
while True:
    a = p.S3G_INPUT(cb.IN_BE_STATUS)
    b = p.S3G_INPUT(cb.IN_APG_STATUS)
    c = p.S3G_INPUT(cb.IN_FIFO_DATA_COUNT)
    d1 = p.S3G_INPUT(cb.IN_MOTOR1_HOLD_X)
    d2 = p.S3G_INPUT(cb.IN_MOTOR2_HOLD_X)
    
    if (a, b, c, d1, d2) != (oa, ob, oc, od1, od2):
        print(a,b,c,d1,d2, od1 - d1, od2 - d2)
        if (d1, d2) != (od1, od2):
            es = p.S3G_INPUT(cb.IN_ES_STATUS)
            if es & 2:
                print("es1", es, d1 - pd1)
                pd1 = d1
                p.S3G_STB(cb.STB_ES_UNLOCK)
            if es & 32:
                print("es1", es, d2 - pd2)
                pd2 = d2
                p.S3G_STB(cb.STB_ES_UNLOCK)
        
    (oa, ob, oc, od1, od2) = (a,b,c,d1,d2) 
    if a & 1 == 0:
        break
    time.sleep(0.001)

BUF_OUTPUT(0, 50000)
BUF_OUTPUT(2, 40)
BUF_OUTPUT(3, 100)
BUF_OUTPUT(4, 400)
BUF_OUTPUT(5, 500)
BUF_OUTPUT(7, -2138996608)
BUF_OUTPUT(13, 500000)
BUF_OUTPUT(1, 5000)
BUF_PARAM_ADDR(0)
BUF_PARAM_WRITE_LO(0, 3)
BUF_PARAM_WRITE_LO(3, 0)
BUF_PARAM_WRITE_LO(1, 500000)
BUF_PARAM_WRITE_LO(1, 0)
BUF_PARAM_WRITE_LO(1, 0)
BUF_PARAM_WRITE_LO(1, 50000000)
BUF_PARAM_WRITE_LO(1, 500000)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_STB(0x00000004)
BUF_WAIT_ALL(0x00000002)
BUF_CLEAR(0x00000002)
BUF_OUTPUT(1, 3000)
BUF_PARAM_WRITE_LO_NC(3)
BUF_PARAM_WRITE_LO(4, -500000)
BUF_PARAM_WRITE_LO(3, 5000000)
BUF_STB(0x00000010)
BUF_WAIT_ALL(0x00000002)
BUF_CLEAR(0x00000002)
BUF_OUTPUT(1, 1000)
BUF_PARAM_ADDR(0)
BUF_PARAM_WRITE_LO(0, 3)
BUF_PARAM_WRITE_LO(4, -500000)
BUF_PARAM_WRITE_LO(3, 0)
BUF_STB(0x00000010)
BUF_WAIT_ALL(0x00000002)
BUF_CLEAR(0x00000002)
BUF_OUTPUT(1, 0)
BUF_S

In [4]:
oa = None
p.S3G_STB(cb.STB_ES_UNLOCK)

while True:
    a = p.S3G_INPUT(cb.IN_ES_STATUS)
    
    if a != oa:
        print(a)
        p.S3G_STB(cb.STB_ES_UNLOCK)
        
    oa = a 
    time.sleep(0.01)

0
48
16
32
0


KeyboardInterrupt: 

In [59]:
from valurap2.buf_commands import CommandBuffer
import time

cb = CommandBuffer()

cb.BUF_OUTPUT(0, 50000),        # step time 0.001s
cb.BUF_OUTPUT(2, 40),           # step_bit = 40
cb.BUF_OUTPUT(3, 100),          # pre_n
cb.BUF_OUTPUT(4, 400),          # pulse_n
cb.BUF_OUTPUT(5, 500),          # post_n
cb.BUF_OUTPUT(7, 0x80988089),   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(13, 500000),       # ES_TIMEOUT 10ms

cb.BUF_OUTPUT(1, 20000),              # steps_val = 20000 (20 seconds)
cb.BUF_PARAM_ADDR(0),
cb.BUF_PARAM_WRITE_LO(0, 3),           # CH0.STATUS=3 (enable + target_v)
cb.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
cb.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.A
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
cb.BUF_PARAM_WRITE_LO(1, 210000000),  # CH0.TARGET_V
cb.BUF_PARAM_WRITE_LO(1, 15000000),     # CH0.ABORT_A

cb.BUF_PARAM_WRITE_LO_NC(3),           # CH0.STATUS=3 (enable + target_v)
cb.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
cb.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.A
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
cb.BUF_PARAM_WRITE_LO(1, 70000000),  # CH0.TARGET_V
cb.BUF_PARAM_WRITE_LO(1, 15000000),     # CH0.ABORT_A

cb.BUF_PARAM_WRITE_LO_NC(0),   # CH2
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH3
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH4
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH5
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH6
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH7
cb.BUF_STB(cb.STB_ASG_START),   # Start ASG cycles

cb.BUF_DONE(0)                # End of program for BE

p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(cb.STB_ASG_ABORT)
p.S3G_STB(cb.STB_ES_UNLOCK)
p.S3G_STB(cb.STB_SP_ZERO)
p.S3G_CLEAR(0xffffffff)
p.S3G_OUTPUT(cb.OUT_MSG_X_VAL, 0)
p.S3G_OUTPUT(cb.OUT_MSG_CONFIG0, 0x82008200)
p.S3G_STB(cb.STB_MSG_SET_X)
p.S3G_OUTPUT(cb.OUT_MSG_CONFIG0, 0x80008000)
p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution

oa, ob, oc, od1, od2 = None, None, None, -1, -1
pd1 = -1
pd2 = -1
oes = None
while True:
    a = p.S3G_INPUT(cb.IN_BE_STATUS)
    b = p.S3G_INPUT(cb.IN_APG_STATUS)
    c = p.S3G_INPUT(cb.IN_FIFO_DATA_COUNT)
    d1 = p.S3G_INPUT(cb.IN_MOTOR1_HOLD_X)
    d2 = p.S3G_INPUT(cb.IN_MOTOR2_HOLD_X)
    ints = p.S3G_INPUT(cb.IN_PENDING_INTS)
    es = p.S3G_INPUT(cb.IN_ES_STATUS)
    
    if (a, b, c, d1, d2, es) != (oa, ob, oc, od1, od2, oes):
        print(a,b,c,d1, d2, od1 - d1, od2 - d2, "{:b}".format(ints))
        if (d1, d2) != (od1, od2):
            if es & 32:
                print("es1", es, d1 - pd1)
                pd1 = d1
                p.S3G_STB(cb.STB_ES_UNLOCK)
            if es & 2:
                print("es2", es, d2 - pd2)
                pd2 = d2
                p.S3G_STB(cb.STB_ES_UNLOCK)
    if ints & (cb.INT_ASG_DONE):
        print("stops are not reached")
        break
    if (ints & (cb.INT_APG0_ABORT_DONE)) and (ints & (cb.INT_APG1_ABORT_DONE)):
        print("Both stops are reached")
        cd1 = p.S3G_INPUT(cb.IN_MOTOR1_X)
        cd2 = p.S3G_INPUT(cb.IN_MOTOR2_X)
        print(pd1, cd1, cd1 - pd1, pd2, cd2, cd2 - pd2)
        break
        
    (oa, ob, oc, od1, od2, oes) = (a,b,c,d1,d2,es) 
    #if a & 1 == 0:
    #    break
    time.sleep(0.001)

0 1 0 103 63 -104 -64 1
0 1 0 3080 63 -2977 0 100000001
es1 48 3081
0 1 0 3080 63 0 0 100000001
0 1 0 3170 63 -90 0 100000001
0 1 0 3170 3111 0 -3048 1100000001
es2 19 3112
Both stops are reached
3080 3170 90 3111 3127 16


In [21]:
from valurap2 import emulate
from valurap2.profile import ProfileSegment

pr_opt = [
    [5, [
       ProfileSegment(apg=0, x=0),
       ProfileSegment(apg=1, x=0),
    ]],
    [1000, [
       ProfileSegment(apg=0, a=1000000, target_v = 100000000),
       ProfileSegment(apg=1, a=5000000, target_v = 100000000),
    ]]
]
apg_states = {}
a = emulate.emulate(pr_opt, apg_states=apg_states, verbose=0, accel_step=50000, spms=[80,80])
str(apg_states[0])

Warning, explicit X in segment
Warning, explicit X in segment


'<ApgState x=53.94 v=56.84 a=0.00 j=0.00 jj=0.00 target_v=56.84>'